In [1]:
random_state = 129763 # In case I need it

try:
    import os
    print(os.getcwd())
    import tensorflow as tf
    print( tf.constant( 'Hello from TensorFlow ' + tf.__version__ ) )
    import pandas as pd
    import numpy as np
    from numpy import array
    from keras.models import Sequential
    from keras.layers import LSTM
    from keras.layers import Dense

    print("✔️ Successfully imported all packages")
except Exception as ex:
    print("❌ Could not import one or more libraries:", ex)

c:\Users\thomas.henno\Desktop\GitRepos\course-machine-learning\week-four\AS14
tf.Tensor(b'Hello from TensorFlow 2.1.0', shape=(), dtype=string)
✔️ Successfully imported all packages
Using TensorFlow backend.


In [2]:
df = pd.read_excel("Dataset.xlsx", skiprows=2, header=None)

df.head()

,0,1,2,3,4,5,6,7,8,9,...,673,674,675,676,677,678,679,680,681,682
0,2521.234,1554.951,1760.839,1874.076,1770.127,2167.416,2250.602,1567.658,2382.911,1786.066,...,1329.016,1056.066,1736.332,1544.154,1555.864,1702.201,1854.153,1646.496,1520.763,1368.026
1,1944.512,1566.987,1224.113,1500.748,3558.259,2831.670,2078.168,2216.085,1804.972,2051.166,...,5592.715,1927.505,2702.186,1364.702,2801.333,2110.587,2153.170,1852.452,1991.153,2549.164
2,2045.401,1691.263,4474.665,2021.804,5016.791,1812.510,3736.011,2855.442,2085.921,1721.896,...,2012.310,1700.974,1708.750,2220.219,1324.229,2571.308,1900.011,2130.460,1793.301,1772.942
3,2165.915,2214.965,2395.277,1909.094,1653.703,2003.161,1731.560,1907.299,1801.705,1906.117,...,1215.273,1127.290,2038.196,1871.442,1468.785,1840.928,1682.228,2194.603,2787.753,1712.142
4,2745.535,2794.596,2250.799,2202.103,1552.819,2111.839,1948.280,1637.539,1694.834,1754.551,...,1832.339,1548.872,1293.723,2272.632,1986.716,1863.857,1858.556,2018.394,1623.119,1658.063


In [3]:
df = df.aggregate("mean", axis="rows")

In [4]:
df.shape

(683,)

In [5]:
# split a univariate sequence into samples
def split_sequence(sequence, n_steps):
    X, y = list(), list()
    for i in range(len(sequence)):
        # find the end of this pattern
        end_ix = i + n_steps
        # check if we are beyond the sequence
        if end_ix > len(sequence)-1:
            break
        # gather input and output parts of the pattern
        seq_x, seq_y = sequence[i:end_ix], sequence[end_ix]
        X.append(seq_x)
        y.append(seq_y)
    return array(X), array(y)

In [6]:
train = array(df[0:504])
test = array(df[504:])

print(train.shape)
print(type(train))
print(test.shape)
print(type(test))

(504,)
<class 'numpy.ndarray'>
(179,)
<class 'numpy.ndarray'>


In [7]:
n_steps = 48
n_features = 1

X_train, y_train = split_sequence(train, n_steps)
X_train

array([[2178.96283333, 2140.82616667, 2029.30108333, ..., 1623.93602778,
        1614.53138889, 1865.56513889],
       [2140.82616667, 2029.30108333, 1878.84486111, ..., 1614.53138889,
        1865.56513889, 1999.42327778],
       [2029.30108333, 1878.84486111, 2105.95708333, ..., 1865.56513889,
        1999.42327778, 2141.1095    ],
       ...,
       [1637.28691667, 1656.06191667, 1909.40827778, ..., 1713.95266667,
        1715.19125   , 1673.01405556],
       [1656.06191667, 1909.40827778, 2074.47794444, ..., 1715.19125   ,
        1673.01405556, 1631.88108333],
       [1909.40827778, 2074.47794444, 2099.55091667, ..., 1673.01405556,
        1631.88108333, 1737.24658333]])

In [8]:
# reshape from [samples, timesteps] into [samples, timesteps, features]
X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], n_features))
X_train.shape

(456, 48, 1)

In [11]:
# Make LSTM model
model = Sequential()
model.add(LSTM(75, activation="relu", input_shape=(n_steps, n_features)))
model.add(Dense(1))
model.compile(optimizer="adam", loss="mse")

# Fit model
model.fit(X_train, y_train, epochs=200, verbose=1, batch_size=5)

Epoch 1/200
456/456 [==============================] - 2s 5ms/step - loss: 3142285.0729
Epoch 2/200
456/456 [==============================] - 2s 5ms/step - loss: 2094688.8940
Epoch 3/200
456/456 [==============================] - 2s 5ms/step - loss: 1048737.3241
Epoch 4/200
456/456 [==============================] - 2s 5ms/step - loss: 19102726.2527
Epoch 5/200
456/456 [==============================] - 2s 5ms/step - loss: 28310449.8438
Epoch 6/200
456/456 [==============================] - 2s 5ms/step - loss: 10393085.4059
Epoch 7/200
456/456 [==============================] - 2s 5ms/step - loss: 1414348.0257
Epoch 8/200
456/456 [==============================] - 2s 5ms/step - loss: 1004479.8793
Epoch 9/200
456/456 [==============================] - 2s 5ms/step - loss: 586789661.4956
Epoch 10/200
456/456 [==============================] - 2s 5ms/step - loss: 94905471.6908
Epoch 11/200
456/456 [==============================] - 2s 5ms/step - loss: 960579610.4030
Epoch 12/200
456/456 [

In [13]:
X_test, y_test = split_sequence(train, n_steps)

In [15]:
# demonstrate prediction
X_test = X_test.reshape((X_test.shape[0], X_test.shape[1], n_features))
y_pred = model.predict(X_test, verbose=0)

[[2000.443 ]
 [1968.4773]
 [1968.9369]
 [1930.3079]
 [1971.304 ]
 [1921.917 ]
 [1855.3833]
 [1826.5803]
 [1790.4589]
 [1783.0203]
 [1806.6191]
 [1796.3975]
 [1770.1565]
 [1729.3242]
 [1703.2075]
 [1692.5734]
 [1663.8191]
 [1650.2821]
 [1652.7761]
 [1658.7466]
 [1691.0599]
 [1739.9304]
 [1792.1807]
 [1848.954 ]
 [1880.8103]
 [1840.5442]
 [1850.7888]
 [1877.7422]
 [1881.8868]
 [1896.4078]
 [1895.5952]
 [1872.364 ]
 [1837.0627]
 [1787.5295]
 [1824.1755]
 [1817.1523]
 [1818.9028]
 [1779.7   ]
 [1768.5645]
 [1773.4877]
 [1759.8853]
 [1738.2107]
 [1714.9115]
 [1709.8699]
 [1700.4384]
 [1766.0638]
 [1825.7706]
 [1914.9159]
 [1979.3674]
 [1989.844 ]
 [1929.9012]
 [1893.342 ]
 [1905.5499]
 [1956.8567]
 [1920.0765]
 [1884.4236]
 [1847.0165]
 [1815.4578]
 [1800.2577]
 [1776.3174]
 [1790.5137]
 [1779.404 ]
 [1801.1012]
 [1807.8386]
 [1813.364 ]
 [1804.1239]
 [1802.6946]
 [1789.2137]
 [1825.9442]
 [1896.2473]
 [1946.7297]
 [2043.4656]
 [2053.079 ]
 [2039.6754]
 [2045.8837]
 [2028.2516]
 [2065.9124]

In [24]:
for real, pred in zip(y_test, y_pred):
    print("Expected: %0.2f Got: %0.2f" % (real, pred[0]))

Expected: 1999.42 Got: 2000.44
Expected: 2141.11 Got: 1968.48
Expected: 2113.68 Got: 1968.94
Expected: 1858.38 Got: 1930.31
Expected: 1857.64 Got: 1971.30
Expected: 2009.21 Got: 1921.92
Expected: 2043.32 Got: 1855.38
Expected: 2038.40 Got: 1826.58
Expected: 1936.25 Got: 1790.46
Expected: 1880.36 Got: 1783.02
Expected: 1807.58 Got: 1806.62
Expected: 1798.63 Got: 1796.40
Expected: 1771.51 Got: 1770.16
Expected: 1697.93 Got: 1729.32
Expected: 1871.14 Got: 1703.21
Expected: 1792.23 Got: 1692.57
Expected: 1848.75 Got: 1663.82
Expected: 1908.77 Got: 1650.28
Expected: 1862.00 Got: 1652.78
Expected: 1802.24 Got: 1658.75
Expected: 1688.37 Got: 1691.06
Expected: 1758.11 Got: 1739.93
Expected: 1853.47 Got: 1792.18
Expected: 2093.31 Got: 1848.95
Expected: 2249.41 Got: 1880.81
Expected: 2042.09 Got: 1840.54
Expected: 2083.16 Got: 1850.79
Expected: 2019.74 Got: 1877.74
Expected: 2123.79 Got: 1881.89
Expected: 2177.57 Got: 1896.41
Expected: 2259.09 Got: 1895.60
Expected: 2216.53 Got: 1872.36
Expected